### Colvolution Neural Network Forward Pass having 2 Convolution Layer and 1 fully connected Layer

In [937]:
import sys
import numpy as np
import pandas as pd
from PIL import Image
import os
import random

### Relu Activation function to achieve Non-linearity

In [938]:
def ReLU(x):
    tempf = []
    for i in x:
        temp1 = []
        for j in i:
            temp = []
            for k in j:
                if k<0 or k>255:
                    temp.append(0)
                else:
                    temp.append(k)
            temp1.append(temp)
        tempf.append(temp1)
    tempf = np.array(tempf)
#     print(tempf.shape)
#     print(tempf)
    return tempf

### Randomly initialize filter matrix 
### filter matrix of dimension 5 * 5

In [939]:
def createFilter(row,col,depth,filtercount):
    filterMatrix = np.random.rand(row,col,depth,filtercount)*0.01
#     filterMatrix = random.random(row,col,depth)
#     filterMatrix=np.full((row,col,depth,filtercount), 1)
    return filterMatrix

### Dot product of input data matrix and filter matrix and sum up all elements of matrix

In [940]:
def convolveOperation(data,dataFilter):
#     print(data.shape)
#     print(dataFilter.shape)
    convRes = np.multiply(data,dataFilter)
    return convRes.sum()

### Perform convolution operation on input data matrix by slicing filter over matrix and generate feature map

In [941]:
def convolve(data,datafilter):
    filterRow, filterCol, filterdepth, filtercount = datafilter.shape
    dataRow, dataCol, datadepth = data.shape
#     print(datadepth)
    convResultRow = dataRow - filterRow + 1
    convResultCol = dataCol - filterCol + 1
    convResult = np.zeros((convResultRow,convResultCol,filtercount))
    for numfil in range(filtercount):
        for x in range(convResultRow):
            for y in range(convResultCol):
                if x+filterRow > dataRow or y+filterCol > dataCol:
                    break
                convResult[x][y][numfil] = convolveOperation(data[x:x+filterRow,y:y+filterCol,:],datafilter[:,:,:,numfil])
    return convResult

### Perform pooling operation over data(Result of relu)

In [942]:
def maxPooling(data,poolRowSize,poolColSize,stride):
    dataRow, dataCol, resLayer = data.shape
    
    poolResCols = int((dataCol - poolColSize)/stride+1)
    poolResRows = int((dataRow - poolRowSize)/stride+1)
    
#     print(type(poolResCols))
#     print(type(poolResRows))
    poolRes = np.zeros((poolResRows,poolResCols,resLayer))
    for l in range(resLayer):
        i = 0
        while i < dataRow:
            j = 0
            while j < dataCol and i+poolRowSize < dataRow and j+poolColSize < dataCol:
                poolRes[int(i/2),int(j/2),l] = np.max(data[i:i+poolRowSize,j:j+poolColSize,l])
                j += stride
            i += stride
    return poolRes

### Create random weight matrix for neural network with one hidden layer

In [943]:
def createWeightMatrix(inputunits,hiddenunits,outputuints):
    hiddenWeights = np.random.randn(inputunits,hiddenunits)
    outputWeights = np.random.randn(hiddenunits,outputuints)
    return hiddenWeights,outputWeights

### Sigmoid activation function for forward pass in neural network

In [944]:
def sigmoid(x):
    x = -x
    return 1 / (1+np.exp(x))

### Forward pass of a neural network

In [945]:
def forwardPass(i,inputunits,hiddemunits,outputunits):
    hiddenWeights, outputWeights = createWeightMatrix(inputunits,hiddemunits,outputunits)
    hypo_1 = np.dot(i,hiddenWeights)
    ypredicted_1 = sigmoid(hypo_1)
    hypo_2 = np.dot(ypredicted_1,outputWeights)
    ypredicted_2 = sigmoid(hypo_2)
    return ypredicted_2

### Apply softmax on a output of a neural network to convert values into a probabilities to find most likelihood

In [946]:
def softMax(z):
    expScores = np.exp(z)
    probs = expScores/np.sum(expScores)
    return probs

In [947]:
filterRows = 5
filterCols = 5
filterdepth = 3

filterDepthLayer_1 = 6
filterDepthLayer_2 = 16

nnInputUnits = 120
nnHiddenUnits = 84
nnOutputUnits = 10

poolingRows = 2
poolingCols = 2
filterDepth_fc = 120


imgPath = "test.jpeg"
imgRows = 32
imgCols = 32
imgdepth = 3

stride = 2

In [948]:
if os.path.isfile(imgPath):
    img = Image.open(imgPath)
    imgArray = np.array(img)
    print("Image Dimension :",imgArray.shape)
    img1 = Image.fromarray(imgArray,'RGB')
    img1 = img1.resize((312,312))
    img1.show()
    img = img.resize((imgRows,imgCols),Image.ANTIALIAS)
    reducedImageArray = np.array(img)
    img1 = Image.fromarray(reducedImageArray,'RGB')
    img1 = img1.resize((312,312))
    img1.show()
    print("Reduced Image Dimension :",reducedImageArray.shape)
else:
    print("File Not Found")
    sys.exit(1)


Image Dimension : (194, 260, 3)
Reduced Image Dimension : (32, 32, 3)


### Actual Implementation

In [949]:
# ------------------------ Pass-1 ---------------------------

# create Filter
filterLayer_1 = createFilter(filterRows,filterCols,filterdepth,filterDepthLayer_1)
# print(filterLayer_1)
print(filterLayer_1.shape)
# perform convolution operation
# print(imageArray.shape)
convResult_1 = convolve(reducedImageArray,filterLayer_1)
print("Dimensions after 1st Convolution :",convResult_1.shape)
# Apply Relu function
reluResult_1 = ReLU(convResult_1)
img1 = Image.fromarray(reluResult_1,'RGB')
img1 = img1.resize((312,312))
img1.show()
poolResult_1 = maxPooling(reluResult_1,poolingRows,poolingCols,stride)
img1 = Image.fromarray(poolResult_1,'RGB')
img1 = img1.resize((312,312))
img1.show()
print("Dimensions after 1st Max Pooling:",poolResult_1.shape)


# ------------------------ Pass-2 ---------------------------

filterLayer_2 = createFilter(filterRows,filterCols,filterDepthLayer_1,filterDepthLayer_2)
print(filterLayer_2.shape)
convResult_2 = convolve(poolResult_1,filterLayer_2)
print("Dimension after 2nd Convolution:",convResult_2.shape)
reluResult_2 = ReLU(convResult_2)
img1 = Image.fromarray(reluResult_2,'RGB')
img1 = img1.resize((312,312))
img1.show()
poolResult_2 = maxPooling(reluResult_2,poolingRows,poolingCols,stride)
img1 = Image.fromarray(poolResult_2,'RGB')
img1 = img1.resize((312,312))
img1.show()
print("Dimension after 2nd Max Pooling:",poolResult_2.shape)

# ------------------------ First FC layer ---------------------------

filterLayer_3 = createFilter(filterRows,filterCols,filterDepthLayer_2,filterDepth_fc)
print(filterLayer_3.shape)
print("Filter Dimensions for FC Convolution:",filterLayer_3.shape)
convResult_3 = convolve(poolResult_2,filterLayer_3)
print("Dimensions after FC Convolution:",convResult_3.shape)
reluResult_3 = ReLU(convResult_3)
img1 = Image.fromarray(reluResult_3,'RGB')
img1 = img1.resize((312,312))
img1.show()
print("Dimensions after applying ReLU:",reluResult_3.shape)

# ------------------- Neural Network Forward Pass ---------------------

nnOutput = forwardPass(reluResult_3,nnInputUnits,nnHiddenUnits,nnOutputUnits)

print(nnOutput.shape)

softmaxRes = softMax(nnOutput[0,0,:])
print("Predicted class of Image: ")
print(np.argmax(softmaxRes))

(5, 5, 3, 6)
Dimensions after 1st Convolution : (28, 28, 6)
Dimensions after 1st Max Pooling: (14, 14, 6)
(5, 5, 6, 16)
Dimension after 2nd Convolution: (10, 10, 16)
Dimension after 2nd Max Pooling: (5, 5, 16)
(5, 5, 16, 120)
Filter Dimensions for FC Convolution: (5, 5, 16, 120)
Dimensions after FC Convolution: (1, 1, 120)
Dimensions after applying ReLU: (1, 1, 120)
(1, 1, 10)
Predicted class of Image: 
2


/home/vatsal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
